In [30]:
import pandas as pd
import numpy as np
import math

In [31]:
def depth2(data):
    #input is a pandas data frame with gene expression values
    dt = data.T
    result = dt.apply(lambda col: (col-col.mean())/col.std())
    result = result.replace(np.nan,0)
    result = result.apply(lambda col: abs(col)).T
    return result.std()

In [110]:
filename = "example.txt"
df = pd.read_csv(filename, sep='\t',index_col=[0])
df.index.names = ['EntrezID']
index_values = df.index.values
print(index_values)
df

[100130426 100133144 100134869 ...     26009    387590    389932]


,TCGA-OR-A5J1-01A-11R-A29S-07,TCGA-OR-A5J2-01A-11R-A29S-07,TCGA-OR-A5J3-01A-11R-A29S-07,TCGA-OR-A5J5-01A-11R-A29S-07,TCGA-OR-A5J6-01A-31R-A29S-07,TCGA-OR-A5J7-01A-11R-A29S-07,TCGA-OR-A5J8-01A-11R-A29S-07,TCGA-OR-A5J9-01A-11R-A29S-07,TCGA-OR-A5JA-01A-11R-A29S-07,TCGA-OR-A5JB-01A-11R-A29S-07,...,TCGA-OR-A5JZ-01A-11R-A29S-07,TCGA-OR-A5K0-01A-11R-A29S-07,TCGA-OR-A5K1-01A-11R-A29S-07,TCGA-OR-A5K2-01A-11R-A29S-07,TCGA-OR-A5K3-01A-11R-A29S-07,TCGA-OR-A5K4-01A-11R-A29S-07,TCGA-OR-A5K5-01A-11R-A29S-07,TCGA-OR-A5K6-01A-11R-A29S-07,TCGA-OR-A5K8-01A-11R-A29S-07,TCGA-OR-A5K9-01A-11R-A29S-07
EntrezID,,,,,,,,,,,,,,,,,,,,,
100130426,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.898170,1.197677,0.000000,0.000000,0.000000,0.000000,0.000000
100133144,2.092918,1.880294,1.448954,0.000000,0.000000,1.115899,1.288181,0.000000,2.447738,3.023131,...,2.550014,1.497485,0.870345,1.193456,2.418649,2.287679,3.809198,0.000000,0.000000,1.079020
100134869,2.304073,3.321178,2.919340,1.350045,2.451778,2.818217,1.717035,1.238726,2.598865,0.000000,...,2.636706,1.975777,0.870345,2.011388,3.444084,2.465034,3.113150,3.012050,3.045600,1.060877
10357,7.230120,6.358918,6.451012,5.779042,6.085474,6.707439,6.583934,6.307347,6.134198,7.291944,...,6.303383,5.586011,6.416019,5.921208,5.114167,6.241377,5.869373,6.933891,6.378617,6.923156
10431,10.990885,10.350856,10.043928,11.199605,10.298047,10.866835,9.960041,10.783913,10.246183,10.970516,...,9.529827,10.098213,10.004238,11.602120,10.533565,9.285444,10.771786,10.712901,10.079854,10.490846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7791,10.847072,11.579783,11.375369,11.210021,9.469787,9.741795,13.415422,11.051995,10.922003,11.984187,...,10.019746,11.920451,10.521644,10.333793,10.183273,9.280253,11.353184,11.041671,10.852221,9.935183
23140,10.178093,10.889494,10.534738,10.156338,9.639107,9.811718,10.098995,9.972436,9.823148,8.536362,...,9.582599,11.420743,8.964383,9.067492,9.824263,9.369719,9.369689,11.180242,9.602676,10.791072
26009,9.221736,9.648604,8.775672,9.005009,8.896596,8.637030,8.855119,9.329175,9.086964,9.960629,...,9.076033,7.675875,8.815886,8.625224,8.788354,9.397551,7.592512,8.572156,8.563610,8.862265


In [ ]:
df = df[ (df['gene_type'] == 'protein_coding')] 
df = df[["gene_name"]]
df

In [136]:
import json

f = open('./metadata.cart.2023-04-06.json')
data = json.load(f)

import pandas as pd
filename = "./gdc/14969d5e-273a-42a5-9bb1-4c585d987f35/51808cda-8154-4dfd-a251-5927cd2d3ef2.rna_seq.augmented_star_gene_counts.tsv"
df = pd.read_csv(filename, sep='\t',skiprows=[0])
df = df[ (df['gene_type'] == 'protein_coding')] 
df = df[["gene_name"]]

for patient in data:
    folder = patient['file_id']
    gene_exp_file = patient['file_name']
    patient_id = patient['associated_entities'][0]['entity_submitter_id']
    path = "gdc/"+folder+"/"+gene_exp_file
    dt = pd.read_csv(path, sep='\t',skiprows=[0])
    dt = dt[ (dt['gene_type'] == 'protein_coding')] 
    dt = dt[["fpkm_uq_unstranded"]]
    dt.columns = [patient_id]
    df = df.join(dt)
df

,gene_name,TCGA-OR-A5K0-01A-11R-A29S-07,TCGA-OR-A5L6-01A-11R-A29S-07,TCGA-OR-A5LT-01A-11R-A29S-07
4,TSPAN6,10.9398,8.1779,18.0612
5,TNMD,0.0000,0.1848,0.0000
6,DPM1,57.1220,53.0300,41.2177
7,SCYL3,0.5686,1.3209,1.4493
8,C1orf112,0.2430,0.6853,0.6470
...,...,...,...,...
60654,AL451106.1,0.0000,0.0000,0.0000
60659,AC008763.4,0.0000,0.0000,0.0000
60661,AC006486.3,0.0000,0.0000,0.0000
60662,AL391628.1,0.0199,0.0189,0.0221
